In [ ]:
import os
file_names:list[str] = os.listdir('stations')
abs_paths:list[str] = []
for name in file_names:
    if '每日各站進出站人數' in name:
        abs_paths.append(os.path.abspath(f'stations/{name}'))
    
abs_paths

In [ ]:
import pandas as pd
from pandas import DataFrame

all_df:list[DataFrame] = [pd.read_csv(path) for path in abs_paths]
display(all_df)


In [ ]:
inOut_df = pd.concat(all_df)
inOut_df.columns = ['乘車日','車站代碼','進站人數','出站人數']
inOut_df

In [ ]:
import requests
url2 = 'https://ods.railway.gov.tw/tra-ods-web/ods/download/dataResource/0518b833e8964d53bfea3f7691aea0ee'
response = requests.get(url2)
if response.status_code == 200:
    station_data = response.json()

station_df = pd.DataFrame(station_data,columns=['stationCode','stationName'])
station_df.columns = ['車站代碼','中文站名']
station_df['車站代碼'] = station_df['車站代碼'].astype(int)
station_df

In [ ]:
station_all_df = pd.merge(inOut_df,station_df,on='車站代碼',how='left')
station_all_df

In [ ]:
station_all_df.info()

In [ ]:
#移除Na
station_all_df1 = station_all_df.dropna()
station_all_df1.info()


In [ ]:
#乘車日變為datetime
station_all_df1['乘車日'] = pd.to_datetime(station_all_df1['乘車日'].astype(str))
station_all_df1.info()

In [ ]:
station_all_df2

In [ ]:
station_all_df2.groupby('中文站名')[['進站人數','出站人數']].agg(['sum','max','min'])

In [ ]:
station_array = station_all_df2.values
station_array[:,0]

In [ ]:
station_array[:,[0,2,3]]

In [23]:
station_all_df2.query('乘車日=="2023"')

/tmp/ipykernel_47535/2084214848.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  station_all_df2.query('乘車日=="2023"')


,乘車日,車站代碼,進站人數,出站人數,中文站名
147238,2023-01-01,6100,19,9,東里
147213,2023-01-01,5070,344,422,南州
147214,2023-01-01,5080,18,25,鎮安
147215,2023-01-01,5090,546,545,林邊
147216,2023-01-01,5100,126,156,佳冬
...,...,...,...,...,...
147140,2023-01-01,3310,1434,1327,五權
147139,2023-01-01,3300,37255,33837,臺中
147138,2023-01-01,3290,1222,1224,精武
147137,2023-01-01,3280,2145,2162,太原
